<a href="https://colab.research.google.com/github/taniaestudar/Few_Shot_Learning/blob/main/cod_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TÓPICOS EM OTIMIZAÇÃO
## PROBLEMA DE META LEARNING E FEW SHOT
 Grupo 2:

EDINEY PEDROZO MATHIAS

MARIANA MIRELA FERREIRA

SARAH RODRIGUES PREZOTTO

TÂNIA CRISTINA VALASKY


---



### **Importações**

In [ ]:
import time
import os
import random
import warnings
import numpy as np
import pandas as pd

# Manipulação de Imagens
from PIL import Image

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Kaggle
import kagglehub

# Scikit-Learn (Machine Learning Tradicional)
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (GridSearchCV, StratifiedKFold, train_test_split)
from sklearn.metrics import (roc_curve, roc_auc_score, confusion_matrix, classification_report)
from sklearn.manifold import TSNE

# TensorFlow / Keras (Deep Learning)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


### **Baixar dataset e configurar caminho**

O dataset que será utilizado é o “[dataset perros, gatos y aves](https://www.kaggle.com/datasets/diegosanand/dataset-perros-gatos-y-aves)”, encontrado na plataforma Kaggle. Essa base de dados contém oito diretórios de imagens - com gatos, cachorros e aves - que são compostos por dez imagens cada.

Esse dataset, é formarmado por:

- 80 imagens divididas em 8 pastas (10 imagens cada):
  - 3 pastas (s1, s4, s7) → 30 imagens de cachorros;
  - 3 pastas (s2, s5, s8) → 30 imagens de gatos;
  - 2 pastas (s3, s6) → 20 imagens de aves.


In [ ]:
# ==============================
# BAIXAR DATASET E DEFINIR CAMINHO
# ==============================
path_base = kagglehub.dataset_download("diegosanand/dataset-perros-gatos-y-aves")
print("Dataset baixado em:", path_base)

# Descobrir a pasta principal
if "Perros_gatos_aves" in os.listdir(path_base):
    caminho_dataset = os.path.join(path_base, "Perros_gatos_aves")
else:
    caminho_dataset = path_base

print("Caminho do dataset a ser usado:", caminho_dataset)

# ==============================
# INSPEÇÃO DAS CLASSES
# ==============================
classes = [d for d in os.listdir(caminho_dataset) if os.path.isdir(os.path.join(caminho_dataset, d))]
classes.sort()

print("\n=== INFORMAÇÕES DO DATASET ===")
print(f"Total de classes: {len(classes)}")
for classe in classes:
    pasta = os.path.join(caminho_dataset, classe)
    n_imgs = len([f for f in os.listdir(pasta) if f.lower().endswith((".jpg",".png",".jpeg"))])
    print(f"Classe: {classe} | Quantidade de imagens: {n_imgs}")

# ==============================
# CARREGAR TODAS AS IMAGENS
# ==============================
def carregar_todas_imagens(caminho_dataset, max_imagens=None):
    imagens = []
    count = 0

    for classe in classes:
        pasta_classe = os.path.join(caminho_dataset, classe)
        arquivos = [f for f in os.listdir(pasta_classe) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

        for arquivo in arquivos:
            caminho_img = os.path.join(pasta_classe, arquivo)
            try:
                img = Image.open(caminho_img).convert("RGB")
                img_array = np.array(img, dtype=np.float32) / 255.0
                imagens.append(img_array)
                count += 1

                if max_imagens is not None and count >= max_imagens:
                    return imagens

            except Exception as e:
                print("Erro ao abrir:", caminho_img, e)

    return imagens

# Exemplo: carregar todas as imagens
imagens = carregar_todas_imagens(caminho_dataset, max_imagens=None)
print(f"\nTotal de imagens carregadas: {len(imagens)}")

In [ ]:
# ==============================
# MOSTRAR IMAGENS ORIGINAIS
# ==============================
def mostrar_imagens_originais(caminho_dataset, samples_per_class=2):
    classes = os.listdir(caminho_dataset)
    plt.figure(figsize=(12, 4*len(classes)))

    for i, classe in enumerate(classes):
        pasta_classe = os.path.join(caminho_dataset, classe)
        arquivos = os.listdir(pasta_classe)
        exemplos = random.sample(arquivos, min(samples_per_class, len(arquivos)))

        for j, arquivo in enumerate(exemplos):
            caminho_img = os.path.join(pasta_classe, arquivo)
            img = Image.open(caminho_img)

            plt.subplot(len(classes), samples_per_class, i*samples_per_class + j + 1)
            plt.imshow(img)
            plt.title(f"{classe}\n{img.size} | {img.mode}")
            plt.axis("off")

    plt.suptitle("Imagens Originais", fontsize=16)
    plt.tight_layout()
    plt.show()

mostrar_imagens_originais(caminho_dataset)

### **EDA: Exploração e Preparação das Imagens**

A EDA (Análise Exploratória de Dados) em imagens serve para entender como os dados estão distribuídos, se existe algum padrão ou problema, e como preparar corretamente o dataset para o modelo.

**Estatísticas globais das imagens originais**

In [ ]:
# ==============================
# CALCULAR MÉDIA E DESVIO-PADRÃO GLOBAL
# ==============================
all_pixels = np.concatenate([img.reshape(-1, 3) for img in imagens], axis=0)

mean_global = np.mean(all_pixels, axis=0)
std_global = np.std(all_pixels, axis=0)

print("\n=== ESTATÍSTICAS GLOBAIS ===")
print("Média global por canal (R,G,B):", mean_global)
print("Desvio-padrão global por canal (R,G,B):", std_global)

# ==============================
# PLOTAR HISTOGRAMA GLOBAL
# ==============================
r = all_pixels[:,0]
g = all_pixels[:,1]
b = all_pixels[:,2]

plt.figure(figsize=(10,5))
plt.hist(r, bins=256, color='red', alpha=0.5, label='R')
plt.hist(g, bins=256, color='green', alpha=0.5, label='G')
plt.hist(b, bins=256, color='blue', alpha=0.5, label='B')
plt.title(f"Histograma Global RGB — Total de pixels: {all_pixels.shape[0]}")
plt.xlabel("Intensidade (0-1)")
plt.ylabel("Frequência")
plt.legend()
plt.show()

#### **Ajustando as imagens (dimensão + RGB + normalização)**

Isso garante que todas as imagens tenham mesmo tamanho e a normalização melhora a estabilidade do treino.

In [ ]:
# ==============================
# AJUSTANDO AS IMAGENS (RGB + 224x224)
# ==============================
def carregar_dataset(caminho_base, tamanho=(128,128)):
    imagens, labels, classes = [], [], []
    pastas = sorted(os.listdir(caminho_base))

    for label, pasta in enumerate(pastas):
        caminho_pasta = os.path.join(caminho_base, pasta)
        arquivos = [f for f in os.listdir(caminho_pasta) if f.lower().endswith((".jpg"))]

        classes.append(pasta)

        for arquivo in arquivos:
            caminho_img = os.path.join(caminho_pasta, arquivo)
            try:
                with Image.open(caminho_img) as img:
                    if img.mode != "RGB":
                        img = img.convert("RGB")
                    img = img.resize(tamanho)
                    img_array = np.array(img, dtype=np.float32) / 255.0  # normalizado
                    imagens.append(img_array)
                    labels.append(label)
            except Exception as e:
                print("Erro:", caminho_img, e)

    return np.array(imagens), np.array(labels), classes

imagens, labels, classes = carregar_dataset(caminho_dataset)
print("Shape das imagens:", imagens.shape)
print("Labels únicos:", np.unique(labels))
print("Classes:", classes)

#### **Estatísticas após ajuste**

In [ ]:
# ==============================
# CALCULAR MÉDIA E DESVIO-PADRÃO GLOBAL APÓS AJUSTE
# ==============================
all_pixels = np.concatenate([img.reshape(-1, 3) for img in imagens], axis=0)

mean_global = np.mean(all_pixels, axis=0)
std_global = np.std(all_pixels, axis=0)

print("\n=== ESTATÍSTICAS GLOBAIS APÓS AJUSTE ===")
print("Média global por canal (R,G,B):", mean_global)
print("Desvio-padrão global por canal (R,G,B):", std_global)

# ==============================
# PLOTAR HISTOGRAMA GLOBAL APÓS AJUSTE
# ==============================
r = all_pixels[:,0]
g = all_pixels[:,1]
b = all_pixels[:,2]

plt.figure(figsize=(10,5))
plt.hist(r, bins=256, color='red', alpha=0.5, label='R')
plt.hist(g, bins=256, color='green', alpha=0.5, label='G')
plt.hist(b, bins=256, color='blue', alpha=0.5, label='B')
plt.title(f"Histograma Global RGB APÓS AJUSTE — Total de pixels: {all_pixels.shape[0]}")
plt.xlabel("Intensidade (0-1)")
plt.ylabel("Frequência")
plt.legend()
plt.show()

In [ ]:
# ==============================
# MOSTRAR IMAGENS AJUSTADAS
# ==============================
def mostrar_imagens_ajustadas(X, y, classes, samples_per_class=3):
    print("\nMostrando imagens com o tamanho e formato ajustados:")
    num_classes = len(classes)
    plt.figure(figsize=(12, 4 * num_classes))

    for i, classe in enumerate(classes):
        indices = np.where(y == i)[0]
        exemplos = np.random.choice(indices, size=min(samples_per_class, len(indices)), replace=False)

        for j, idx in enumerate(exemplos):
            plt.subplot(num_classes, samples_per_class, i * samples_per_class + j + 1)
            plt.imshow(X[idx])
            plt.title(f"{classe}\nshape={X[idx].shape}")
            plt.axis("off")

    plt.tight_layout()
    plt.show()

#### **Criação de subconjuntos embaralhados**
Nesta etapa, as imagens são embaralhadas e depois divididas em subconjuntos menores. Ou seja, cada subconjunto recebe uma mistura aleatória de cães, gatos e aves, isso impede que um subconjunto fique dominado por uma única classe.

In [ ]:
# ==============================
# FUNÇÃO PARA CRIAR SUBCONJUNTOS EM MEMÓRIA
# ==============================
def criar_subconjuntos_memoria(imagens, labels, classes, num_subconjuntos=8, tam_subconjunto=10):
    total_imgs = imagens.shape[0]
    indices = list(range(total_imgs))
    random.shuffle(indices)  # embaralhar todas as imagens

    subconjuntos = []

    for i in range(num_subconjuntos):
        start = i * tam_subconjunto
        end = start + tam_subconjunto
        if end > total_imgs:  # caso não tenha imagens suficientes
            end = total_imgs
        subset_idx = indices[start:end]
        subconjuntos.append((imagens[subset_idx], labels[subset_idx]))

    return subconjuntos

# ==============================
# FUNÇÃO PARA PLOTAR SUBCONJUNTO
# ==============================
def plot_subconjunto_memoria(imagens_sub, labels_sub, classes, max_cols=5):
    n = imagens_sub.shape[0]
    cols = min(max_cols, n)
    rows = (n + cols - 1) // cols

    plt.figure(figsize=(15, rows * 3))
    for i in range(n):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(imagens_sub[i])
        plt.title(classes[labels_sub[i]], fontsize=10)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# ==============================
# CRIAR SUBCONJUNTOS
# ==============================
subconjuntos = criar_subconjuntos_memoria(imagens, labels, classes)

# ==============================
# PLOTAR TODOS OS SUBCONJUNTOS
# ==============================
for i, (imgs_sub, labels_sub) in enumerate(subconjuntos):
    print(f"Subconjunto {i+1} - Total de imagens: {imgs_sub.shape[0]}")
    plot_subconjunto_memoria(imgs_sub, labels_sub, classes)

#### **Divisão treino / validação / teste (70% / 15% / 15%)**

Após os subconjuntos, as imagens são divididas em três partes essenciais:

*   **Treino:** usado para ajustar os pesos do modelo;
*   **Validação:** usado para ajustar hiperparâmetros;
*   **Teste:** usado apenas para medir desempenho final.

In [ ]:
# ==============================
# DIVIDIR EM TREINO/VALIDAÇÃO/TESTE
# ==============================
# 70% treino, 15% validação, 15% teste
X_train, X_temp, y_train, y_temp = train_test_split(imagens, labels, test_size=0.3, stratify=labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print("\nTamanhos dos conjuntos:")
print("Treino:", X_train.shape, " Validação:", X_val.shape, " Teste:", X_test.shape)


# ==============================
# MOSTRAR IMAGENS AJUSTADAS
# ==============================
def mostrar_imagens_ajustadas(X_train, y_train, classes, samples_per_class=3):

    print("\nMostrando imagens com o tamanho e formato ajustados:")
    num_classes = len(classes)
    plt.figure(figsize=(12, 4*num_classes))

    for i, classe in enumerate(classes):
        indices = np.where(y_train == i)[0]
        exemplos = np.random.choice(indices, size=min(samples_per_class, len(indices)), replace=False)

        for j, idx in enumerate(exemplos):
            plt.subplot(num_classes, samples_per_class, i*samples_per_class + j + 1)
            plt.imshow(X_train[idx])
            plt.title(f"{classe}\nshape={X_train[idx].shape}")
            plt.axis("off")

    plt.tight_layout()
    plt.show()

mostrar_imagens_ajustadas(X_train, y_train, classes)



---


### **Resultados Preliminares**

#### **Configuração Inicial**  
Fixamos seeds para garantir reprodutibilidade dos resultados.


In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

#### **Construção da CNN Base**  
Conforme descrito no artigo (Moresco, 2022):

- Modelo usado: **VGG16 pré-treinada no ImageNet**;
- Todas as camadas convolucionais são **congeladas** (*transfer learning*);
- A saída é compactada por **GlobalAveragePooling2D**;
- Última camada é uma **densa de 128 neurônios**, produzindo o *embedding* usado pela SNN


In [ ]:
def criar_cnn_base(input_shape=(128, 128, 3), embedding_dim=128):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Congelar camadas convolucionais
    for layer in base_model.layers:
        layer.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(embedding_dim, activation='relu')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

#### **Construção da Rede Neural Siamesa (SNN)**
A SNN recebe duas imagens:

- Extrai *embeddings* das duas imagens  
- Calcula distância absoluta (L1)  
- Passa por uma camada sigmoid para obter similaridade ∈ [0, 1]  

Essa arquitetura é usada para verificar se duas imagens são da mesma classe.


In [ ]:
# ==============================
# MODELO SIAMESE
# ==============================
def criar_modelo_siamese(input_shape=(128, 128, 3)):
    base_cnn = keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation="relu")
    ])

    input_a = keras.Input(shape=input_shape)
    input_b = keras.Input(shape=input_shape)

    encoded_a = base_cnn(input_a)
    encoded_b = base_cnn(input_b)

    distance = layers.Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([encoded_a, encoded_b])
    outputs = layers.Dense(1, activation="sigmoid")(distance)

    model = keras.Model(inputs=[input_a, input_b], outputs=outputs)
    return model


#### **Gerador de Batches (Algoritmo 1)**

Cada batch contém:
- 50% pares **positivos** (mesma classe → label = 1)  
- 50% pares **negativos** (classes diferentes → label = 0)


In [ ]:
def gerar_batch(X, y, batch_size=64):
    metade = batch_size // 2
    X1, X2, labels = [], [], []
    classes = np.unique(y)

    for _ in range(metade):
        cls = random.choice(classes)
        idx = np.where(y == cls)[0]

        if len(idx) < 2:
            i1, i2 = np.random.choice(idx, 2, replace=True)
        else:
            i1, i2 = np.random.choice(idx, 2, replace=False)

        X1.append(X[i1])
        X2.append(X[i2])
        labels.append(1)

        cls1, cls2 = np.random.choice(classes, 2, replace=False)
        i1 = np.random.choice(np.where(y == cls1)[0])
        i2 = np.random.choice(np.where(y == cls2)[0])
        X1.append(X[i1])
        X2.append(X[i2])
        labels.append(0)

    return np.array(X1), np.array(X2), np.array(labels)

#### **Treinamento do Modelo Siames**
Treinamento manual époco-a-época usando `train_on_batch`.  
Mostramos o progresso a cada 10 épocas.


In [ ]:
# ==============================
# CRIAR E COMPILAR MODELO
# ==============================
snn = criar_modelo_siamese()
snn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
EPOCHS = 50
BATCH_SIZE = 64

for epoch in range(EPOCHS):
    X1_batch, X2_batch, y_batch = gerar_batch(X_train, y_train, batch_size=BATCH_SIZE)
    loss, acc = snn.train_on_batch([X1_batch, X2_batch], y_batch)
    if epoch % 10 == 0:
        print(f"Época {epoch}/{EPOCHS} - Loss: {loss:.4f} - Acurácia: {acc:.4f}")


#### **Loss vs Acurácia**

  - ***Loss*** mede quão perto a previsão está do rótulo verdadeiro. Então quanto menor o valor do loss, melhor é o resultado;
  - ***Acurácia*** mede quantos pares foram classificados corretamente.

Para interpretarmos os resultados da célula anterior, podemos observar que, a partir da época 10, o modelo já começa a aprender padrões, pois a loss diminui e a acurácia melhora em relação à época 0. No início (época 0), o modelo estava praticamente aleatório, com uma loss muito alta. Já na época 30, ele começou a capturar o conceito de similaridade entre imagens, e na época 40 mostrou progresso contínuo. Cada vez que a loss diminui e a acurácia aumenta, isso indica que o modelo está aprendendo melhor a diferenciar as classes.

#### **Predição de similaridade**

A Função de Análise por Classe mostra:

1. Imagens da classe no treino  
2. Define uma imagem de referência no teste  
3. Compara com todas as imagens do teste  
4. Ordena por similaridade  
5. Mostra visualização lado a lado  


In [2]:
print("Classes disponíveis no dataset:")
print(classes)

def analisar_classe(classe_desejada, X_train, y_train, X_test, y_test, classes, modelo_siamese, max_imgs=20, imagens_por_tela=10, limiar=0.5):
    """
    Analisa uma classe específica:
    1. Mostra imagens dessa classe no conjunto de treino
    2. Usa uma imagem do conjunto de teste como referência
    3. Prediz similaridade com todas as imagens do teste
    4. Mostra os resultados organizados
    """

    # ==============================
    # 1. Mostrar imagens da classe no treino
    # ==============================
    indice_classe = classes.index(classe_desejada)
    indices = np.where(y_train == indice_classe)[0]
    imgs_classe = X_train[indices]
    total_imgs = len(imgs_classe)

    print(f"Total de imagens da classe '{classe_desejada}' no treino: {total_imgs}")

    max_imgs = min(total_imgs, max_imgs)
    plt.figure(figsize=(15, 3 * (max_imgs // 5 + 1)))
    for i in range(max_imgs):
        plt.subplot((max_imgs // 5) + 1, 5, i + 1)
        plt.imshow(imgs_classe[i])
        plt.title(f"{classe_desejada} - {i+1}")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

    # ==============================
    # 2. Escolher uma imagem de referência no teste
    # ==============================
    idx_ref = np.where(y_test == indice_classe)[0][0]
    img_ref = X_test[idx_ref]

    # ==============================
    # 3. Predizer similaridade com todas as imagens do teste
    # ==============================
    similaridades = []
    for i in range(len(X_test)):
        pred = modelo_siamese.predict([img_ref[np.newaxis, ...], X_test[i][np.newaxis, ...]])[0][0]
        similaridades.append(pred)
    similaridades = np.array(similaridades)

    # ==============================
    # 4. Ordenar e mostrar resultados
    # ==============================
    indices_ordenados = np.argsort(similaridades)[::-1]
    num_telas = int(np.ceil(len(X_test) / imagens_por_tela))

    for t in range(num_telas):
        start = t * imagens_por_tela
        end = min(start + imagens_por_tela, len(X_test))
        subset_idx = indices_ordenados[start:end]

        plt.figure(figsize=(12, 3 * len(subset_idx)))
        for i, idx in enumerate(subset_idx):
            img_cmp = X_test[idx]
            pred = similaridades[idx]
            resposta = f"É {classe_desejada}" if pred > limiar else f"Não é {classe_desejada}"

            # Mostrar lado a lado
            plt.subplot(len(subset_idx), 2, 2*i + 1)
            plt.imshow(img_ref)
            plt.title(f"Imagem de Referência ({classe_desejada})")
            plt.axis("off")

            plt.subplot(len(subset_idx), 2, 2*i + 2)
            plt.imshow(img_cmp)
            plt.title(f"{resposta}\nClasse real: {classes[y_test[idx]]} | Similaridade: {pred:.2f}")
            plt.axis("off")

        plt.tight_layout()
        plt.show()


In [ ]:
# Rodando análise para todas as classes

for c in classes:
  print(f"\n=== Analisando a classe {c} ===")
  analisar_classe(c, X_train, y_train, X_test, y_test, classes, snn)

#### **Uma breve dos resultados obtidos até aqui**
Uma observação importante é que, ao comparar a imagem de referência com ela mesma, seria esperado obter uma similaridade muito próxima de 1, já que se trata da mesma imagem. No entanto, o modelo retornou apenas **0,5**, o que indica que ele ainda não aprendeu a produzir embeddings suficientemente próximos para imagens idênticas. Nossa equipe identificou dois fatores principais que podem explicar esse comportamento:

1. **Treinamento insuficiente:** o modelo foi atualizado apenas 50 vezes usando `train_on_batch`, o que é muito pouco para que aprenda a estruturar adequadamente o espaço de embeddings.
2. **Dataset reduzido:** com poucas amostras por classe, a rede não consegue generalizar bem, o que prejudica a formação de representações discriminativas.

Ao observar as demais predições, percebe-se que os valores de similaridade apresentam certa aleatoriedade. Por exemplo, um gato atingiu similaridade 0,79 em relação a um cachorro — um claro falso positivo, evidenciando que o modelo ainda não diferencia bem as classes. Isso é esperado em cenários com poucos dados e pouco treinamento, onde embeddings de classes distintas acabam ficando próximos.

Além disso, tanto para a classe S1 quanto para as demais, a maioria das imagens recebe valores de similaridade muito baixos. O modelo acerta pouco e parece saturar sua saída (valores muito altos ou muito próximos de zero), sem apresentar uma transição clara entre classes semelhantes e distintas.

Em síntese, o modelo ainda **não é confiável**. Não é necessário analisar todas as predições para perceber isso: o simples fato de uma imagem ser comparada com ela mesma e obter similaridade 0,5 já evidencia que a representação aprendida não está adequada.

Para melhorar o desempenho nas próximas etapas, planejamos:

* **Aumentar significativamente o tempo de treinamento**, gerando mais batches com exemplos positivos e negativos;
* **Utilizar múltiplas imagens de referência por classe** e calcular a média das similaridades, reduzindo a variabilidade;
* **Expandir o dataset**, quando possível, sem descaracterizar a proposta de meta-learning;
* **Inspecionar os *embeddings* gerados**, avaliando se há separação entre classes.



---




### **Melhorando o Modelo**

#### **Aumento de Épocas e Tamanho do Batch**

Agora iniciamos a fase de aprimoramento do modelo, começando pelo aumento do número de épocas e do tamanho do batch.  
Isso significa, na prática, oferecer mais tempo de treinamento ao modelo e fornecer mais exemplos positivos e negativos a cada batch, permitindo que a rede aprenda melhor o conceito de similaridade.

Em vez de testar manualmente cada combinação, vamos definir listas de valores maiores que 50 épocas e maiores que 64 imagens por batch.  
Para cada configuração, registraremos a *training loss* e a *training accuracy*, além de calcular o desempenho final médio no conjunto de validação.


In [ ]:
epoch_list = [60, 110, 160]
batch_list = [128, 256, 512]

VAL_N_BATCHES = 20

def avaliar_em_batches(model, X, y, batch_size, n_batches=VAL_N_BATCHES):
    losses, accs = [], []
    for _ in range(n_batches):
        X1b, X2b, yb = gerar_batch(X, y, batch_size=batch_size)
        loss, acc = model.test_on_batch([X1b, X2b], yb)
        losses.append(loss)
        accs.append(acc)
    return float(np.mean(losses)), float(np.mean(accs))

results = {}

for BATCH_SIZE in batch_list:
    for EPOCHS in epoch_list:
        run_name = f"ep{EPOCHS}_bs{BATCH_SIZE}"
        print("\n" + "="*60)
        print(f"Rodando: {run_name} — {time.strftime('%Y-%m-%d %H:%M:%S')}")

        model = criar_modelo_siamese(input_shape=(128,128,3))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        steps_per_epoch = max(1, len(X_train) // max(1, BATCH_SIZE))

        hist = {'loss': [], 'acc': []}

        for epoch in range(1, EPOCHS + 1):
            epoch_loss, epoch_acc = 0.0, 0.0
            for step in range(steps_per_epoch):
                X1_batch, X2_batch, y_batch = gerar_batch(X_train, y_train, batch_size=BATCH_SIZE)
                loss, acc = model.train_on_batch([X1_batch, X2_batch], y_batch)
                epoch_loss += loss
                epoch_acc += acc
            epoch_loss /= steps_per_epoch
            epoch_acc /= steps_per_epoch

            hist['loss'].append(epoch_loss)
            hist['acc'].append(epoch_acc)

            if epoch % 10 == 0 or epoch == 1 or epoch == EPOCHS:
                print(f"Run {run_name} — Época {epoch}/{EPOCHS} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f}")

        val_loss, val_acc = avaliar_em_batches(model, X_val, y_val, batch_size=BATCH_SIZE, n_batches=VAL_N_BATCHES)
        print(f"Resultado final (val médio): Loss={val_loss:.4f}, Acc={val_acc:.4f}")

        results[run_name] = {
            'EPOCHS': EPOCHS,
            'BATCH_SIZE': BATCH_SIZE,
            'history': hist,
            'val_loss': val_loss,
            'val_acc': val_acc,
            'model': model #isso serve pra guardar o modelo, mas pode encher a memória
        }

#### **Visualizando o Histórico de Treinamento**

Para cada combinação de épocas e batch size, vamos gerar gráficos de loss e acurácia, permitindo visualizar o comportamento do modelo ao longo das iterações.

Isso ajuda a identificar tendências como:
- convergência lenta ou rápida
- overfitting
- instabilidade do treinamento


In [ ]:
for run_name, info in results.items():
    hist = info['history']
    epochs = len(hist['loss'])
    plt.figure(figsize=(10,3))
    plt.suptitle(run_name)

    plt.subplot(1,2,1)
    plt.plot(range(1, epochs+1), hist['loss'])
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.grid(True)

    plt.subplot(1,2,2)
    plt.plot(range(1, epochs+1), hist['acc'])
    plt.xlabel('Época')
    plt.ylabel('Acurácia')
    plt.grid(True)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


#### **Salvando os Resultados Consolidado em um DataFrame**


In [ ]:
# Cria uma lista com os resultados resumidos
dados = []
for run_name, info in results.items():
    dados.append({
        "Execução": run_name,
        "Épocas": info["EPOCHS"],
        "Batch_Size": info["BATCH_SIZE"],
        "Val_Loss": info["val_loss"],
        "Val_Acc": info["val_acc"]
    })

# Converte em DataFrame e salva
df_resultados = pd.DataFrame(dados)
df_resultados.to_csv("resultados_treinos.csv", index=False)

print("\n Resultados salvos em 'resultados_treinos.csv'")
display(df_resultados)

#### **Avaliação Qualitativa dos Modelos**

Agora, vamos examinar visualmente como cada modelo se comporta ao comparar uma imagem de referência com outras imagens do conjunto de teste.

Isso permite observar:
- se a similaridade está coerente;
- se as classes estão mais separadas;
- se há evolução em relação ao modelo inicial.


In [ ]:
# Enumerar modelos de 1 a 9 conforme a ordem da execução
modelos_numerados = {}

for i, (run_name, info) in enumerate(results.items(), start=1):
    nome_modelo = f"modelo_{i}_{run_name}"
    modelos_numerados[nome_modelo] = info['model']
    print(f"{i}. {run_name} — Val_Loss={info['val_loss']:.4f} — Val_Acc={info['val_acc']:.4f}")

In [ ]:
def comparar_modelos_por_classe(classe_desejada, X_train, y_train, X_test, y_test, classes, resultados, max_imgs=5, limiar=0.5):
    """
    Compara visualmente os modelos treinados (armazenados em 'resultados')
    para uma classe específica, mostrando lado a lado o desempenho.
    """
    indice_classe = classes.index(classe_desejada)
    idx_ref = np.where(y_test == indice_classe)[0][0]
    img_ref = X_test[idx_ref]

    print(f"\nComparando modelos para a classe: {classe_desejada}")
    print(f"Imagem de referência — Classe real: {classe_desejada}")
    plt.figure(figsize=(2, 2))
    plt.imshow(img_ref)
    plt.axis("off")
    plt.show()

    # Inclui a imagem de referência e outras aleatórias
    indices_aleatorios = np.concatenate(([idx_ref], np.random.choice(len(X_test), max_imgs - 1, replace=False)))

    # Para cada modelo
    for i, (run_name, info) in enumerate(resultados.items(), 1):
        modelo = info["model"]
        preds = []

        for idx in indices_aleatorios:
            pred = modelo.predict([img_ref[np.newaxis, ...], X_test[idx][np.newaxis, ...]])[0][0]
            preds.append((idx, pred))

        print(f"\n=== Modelo {i} — {run_name} ===")
        plt.figure(figsize=(14, 3))
        for j, (idx, pred) in enumerate(preds):
            plt.subplot(1, max_imgs, j + 1)
            plt.imshow(X_test[idx])
            pred_classe = classes[y_test[idx]]
            titulo = f"{pred_classe}\nSimilaridade: {pred:.2f} → {'É Similar' if pred > limiar else 'Não é'}"
            plt.title(titulo)
            plt.axis("off")

        plt.suptitle(f"Modelo {i}: {run_name}")
        plt.tight_layout()
        plt.show()


In [ ]:
# Comparação para todas as classes:

for c in classes:
    comparar_modelos_por_classe(c, X_train, y_train, X_test, y_test, classes, results)


#### **Ajuste no código**

Agora ajustamos a função de comparação entre modelos para garantir:

- Normalização coerente das imagens antes da predição;

- Impressão da auto-similaridade, que é o caso mais crítico;

- Comparação mais limpa entre modelos, com média das outras similaridades.

Isso nos permite identificar de forma clara que todos os modelos ainda falham em gerar similaridade ≈ 1 para a mesma imagem, o que já indica que a perda usada (*binary crossentropy*) não está orientando bem os *embeddings*.

In [ ]:
def comparar_modelos_por_classe(classe_desejada, X_train, y_train, X_test, y_test, classes, resultados, max_imgs=5, limiar=0.5):
    """
    Compara visualmente os modelos treinados para uma classe específica,
    mostrando lado a lado o desempenho de cada um.
    Agora garante normalização correta e exibe a auto-similaridade da imagem.
    """
    indice_classe = classes.index(classe_desejada)

    # Selecionar imagem de referência
    idx_ref = np.where(y_test == indice_classe)[0][0]
    img_ref = X_test[idx_ref].astype(np.float32)

    print(f"\nComparando modelos para a classe: {classe_desejada}")
    plt.figure(figsize=(2, 2))
    plt.imshow(img_ref)
    plt.title(f"Imagem de referência — {classe_desejada}")
    plt.axis("off")
    plt.show()

    # Selecionar outras imagens aleatórias (de classes variadas)
    indices_aleatorios = np.random.choice(len(X_test), max_imgs - 1, replace=False)
    indices_aleatorios = np.concatenate(([idx_ref], indices_aleatorios))

    for i, (run_name, info) in enumerate(resultados.items(), 1):
        modelo = info["model"]

        print(f"\n=== Modelo {i} — {run_name} ===")

        preds = []
        for idx in indices_aleatorios:
            img_comp = X_test[idx].astype(np.float32)
            pred = modelo.predict([img_ref[np.newaxis, ...], img_comp[np.newaxis, ...]], verbose=0)[0][0]
            preds.append((idx, pred))

        # Mostrar valor da auto-similaridade
        pred_auto = preds[0][1]
        print(f"Similaridade da imagem consigo mesma: {pred_auto:.4f}")
        print(f"Média de similaridade com outras imagens: {np.mean([p for _, p in preds[1:]]):.4f}")

        # Plotagem
        plt.figure(figsize=(15, 3))
        for j, (idx, pred) in enumerate(preds):
            plt.subplot(1, max_imgs, j + 1)
            plt.imshow(X_test[idx])
            pred_classe = classes[y_test[idx]]
            titulo = f"{pred_classe}\nSimilaridade: {pred:.2f} → {'É similar' if pred > limiar else 'Não é'}"
            plt.title(titulo, fontsize=9)
            plt.axis("off")

        plt.suptitle(f"Modelo {i}: {run_name}", fontsize=12, y=1.05)
        plt.tight_layout()
        plt.show()


In [ ]:
for c in classes:
    comparar_modelos_por_classe(c, X_train, y_train, X_test, y_test, classes, results)

#### **Análise:**

Mesmo após os ajustes realizados, a imagem de referência ainda não apresenta similaridade 1 consigo mesma. O modelo continua produzindo valores próximos de 0.5, mesmo quando comparamos duas imagens idênticas.

Isso indica que a rede não está calibrando corretamente a saída da sigmoide.
Esse comportamento é comum quando usamos *binary crossentropy* em modelos siameses baseados em *embeddings*: em vez de aprender distâncias reais, a rede tende a adotar uma saída "neutra".

Uma alternativa mais apropriada para redes siamesas é a contrastive loss, pois ela estrutura diretamente o espaço de embeddings.
Com ela, o modelo é incentivado a:

- Aproximar embeddings quando as amostras pertencem à mesma classe;

- Afastar embeddings quando as amostras são de classes diferentes.

In [ ]:

# -------------------------------------------------------------
# 1. Função de perda contrastiva
# -------------------------------------------------------------
def contrastive_loss(y_true, y_pred, margin=1.0):
    """
    y_true: 1 = diferentes, 0 = iguais
    y_pred: distância euclidiana entre embeddings
    """
    squared_pred = K.square(y_pred)
    margin_term = K.square(K.maximum(margin - y_pred, 0))
    return K.mean((1 - y_true) * squared_pred + y_true * margin_term)

# -------------------------------------------------------------
# 2. Distância Euclidiana
# -------------------------------------------------------------
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

# -------------------------------------------------------------
# 3. Modelo base (gera embeddings)
# -------------------------------------------------------------
def criar_modelo_base(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dense(128, activation='relu')(x)
    embedding = layers.Dense(64)(x)
    embedding = layers.Lambda(lambda x: K.l2_normalize(x, axis=1))(embedding)

    return models.Model(inputs, embedding)

# -------------------------------------------------------------
# 4. Rede Siamesa completa
# -------------------------------------------------------------
def criar_rede_siamese(input_shape):
    base_model = criar_modelo_base(input_shape)
    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)

    embedding_a = base_model(input_a)
    embedding_b = base_model(input_b)

    distancia = layers.Lambda(euclidean_distance)([embedding_a, embedding_b])
    siamese_model = models.Model(inputs=[input_a, input_b], outputs=distancia)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    siamese_model.compile(loss=contrastive_loss, optimizer=optimizer)
    return siamese_model

# ==============================
# 5. GERAR PARES PARA O MODELO SIAMESE
# ==============================
def gerar_pares_hard(X, y, n_pairs_per_class=10):
    """
    Gera pares (positivos e negativos) para treino de rede siamesa.
    - 0 = pares da mesma classe
    - 1 = pares de classes diferentes

    Deixa robusto para o caso de alguma classe ter poucas amostras
    (por exemplo, no conjunto de validação ou teste).
    """
    pares_a, pares_b, labels = [], [], []

    y = np.array(y)
    classes = np.unique(y)
    idx_por_classe = {c: np.where(y == c)[0] for c in classes}

    # Só usa classes com pelo menos 2 amostras (para formar pares positivos)
    classes_validas = [c for c in classes if len(idx_por_classe[c]) >= 2]

    if len(classes_validas) == 0:
        raise ValueError(
            "Nenhuma classe tem pelo menos 2 amostras para gerar pares positivos. "
            "Verifique o conjunto passado para gerar_pares_hard."
        )
    if len(classes_validas) == 1:
        raise ValueError(
            "Apenas uma classe tem pelo menos 2 amostras; "
            "não é possível gerar pares negativos entre classes diferentes."
        )

    for c in classes_validas:
        idx_c = idx_por_classe[c]

        for _ in range(n_pairs_per_class):
            # -------- Pares positivos (mesma classe) --------
            i, j = np.random.choice(idx_c, 2, replace=False)
            pares_a.append(X[i])
            pares_b.append(X[j])
            labels.append(0)  # 0 = iguais

            # -------- Pares negativos (classes diferentes) --------
            classes_negativas = [cn for cn in classes_validas if cn != c]
            c_neg = np.random.choice(classes_negativas)
            idx_c_neg = idx_por_classe[c_neg]

            i = np.random.choice(idx_c)
            j = np.random.choice(idx_c_neg)
            pares_a.append(X[i])
            pares_b.append(X[j])
            labels.append(1)  # 1 = diferentes

    return np.array(pares_a), np.array(pares_b), np.array(labels)

# Gerar pares para treino, validação e teste
pairs_train_a, pairs_train_b, y_train_pairs = gerar_pares_hard(X_train, y_train)
pairs_val_a, pairs_val_b, y_val_pairs = gerar_pares_hard(X_val, y_val)
pairs_test_a, pairs_test_b, y_test_pairs = gerar_pares_hard(X_test, y_test)

print("Pares gerados:")
print("Treino:", pairs_train_a.shape, pairs_train_b.shape)
print("Validação:", pairs_val_a.shape, pairs_val_b.shape)
print("Teste:", pairs_test_a.shape, pairs_test_b.shape)

# -------------------------------------------------------------
# 6. Função para converter distância → similaridade
# -------------------------------------------------------------
def prever_similaridade(model, img1, img2):
    dist = model.predict([np.expand_dims(img1, 0), np.expand_dims(img2, 0)])
    sim = np.exp(-dist)  # converte distância para similaridade (0 a 1)
    return sim

def comparar_modelos_por_classe(classe_desejada, X_test, y_test, classes, resultados,
                                max_imgs=5, mostrar_exemplos=True):
    """
    Compara visualmente os modelos treinados (armazenados em 'resultados')
    para uma classe específica, mostrando a separação das distâncias entre pares
    e exemplos visuais de comparação (iguais e diferentes).
    """
    print(f"\nComparando modelos para a classe: {classe_desejada}")

    # Seleciona índices da classe e de outras classes
    idx_classe = classes.index(classe_desejada)
    idx_positivos = np.where(y_test == idx_classe)[0]
    idx_negativos = np.where(y_test != idx_classe)[0]

    if len(idx_positivos) < 2:
        print(f"Poucas imagens na classe {classe_desejada} para comparação.")
        return

    # Seleciona subconjuntos de imagens
    X_classe = X_test[np.random.choice(
        idx_positivos, min(max_imgs, len(idx_positivos)), replace=False
    )]
    X_outras = X_test[np.random.choice(
        idx_negativos, min(max_imgs, len(idx_negativos)), replace=False
    )]

    # Loop sobre os modelos
    for i, (run_name, info) in enumerate(resultados.items(), start=1):
        model = info["model"]
        print(f"\n=== Modelo {i} — {run_name} ===")

        dist_pos, dist_neg = [], []

        # Calcula distâncias para pares da mesma classe
        for a in X_classe:
            for b in X_classe:
                if not np.array_equal(a, b):
                    d = model.predict([a[None, ...], b[None, ...]], verbose=0)[0][0]
                    dist_pos.append(d)

            # Calcula distâncias para pares de classes diferentes
            for b in X_outras:
                d = model.predict([a[None, ...], b[None, ...]], verbose=0)[0][0]
                dist_neg.append(d)

        # --- HISTOGRAMA ---
        plt.figure(figsize=(8, 5))
        plt.hist(dist_pos, bins=15, alpha=0.6, label='Mesma Classe', color='green')
        plt.hist(dist_neg, bins=15, alpha=0.6, label='Classes Diferentes', color='red')
        plt.xlabel("Distância (menor = mais similar)")
        plt.ylabel("Frequência")
        plt.title(f"Distribuição de Distâncias — {classe_desejada}\n{run_name}")
        plt.legend()
        plt.show()

        print(f"Distância média — Mesma classe: {np.mean(dist_pos):.3f}")
        print(f"Distância média — Diferentes: {np.mean(dist_neg):.3f}")

        # --- EXEMPLOS VISUAIS ---
        if mostrar_exemplos:
            fig, axes = plt.subplots(2, 3, figsize=(10, 6))
            fig.suptitle(f"Exemplos de Comparação — {classe_desejada} ({run_name})",
                         fontsize=14)

            # Pares da mesma classe
            for j in range(3):
                idx_a, idx_b = np.random.choice(range(len(X_classe)), 2, replace=False)
                a, b = X_classe[idx_a], X_classe[idx_b]
                dist = model.predict([a[None, ...], b[None, ...]], verbose=0)[0][0]
                img_pair = np.hstack((np.squeeze(a), np.squeeze(b)))
                axes[0, j].imshow(img_pair, cmap='gray')
                axes[0, j].set_title(f"Iguais — dist={dist:.3f}")
                axes[0, j].axis("off")

            # Pares de classes diferentes
            for j in range(3):
                a = X_classe[np.random.choice(len(X_classe))]
                b = X_outras[np.random.choice(len(X_outras))]
                dist = model.predict([a[None, ...], b[None, ...]], verbose=0)[0][0]
                img_pair = np.hstack((np.squeeze(a), np.squeeze(b)))
                axes[1, j].imshow(img_pair, cmap='gray')
                axes[1, j].set_title(f"Dif. — dist={dist:.3f}")
                axes[1, j].axis("off")

            plt.tight_layout()
            plt.show(block=True)


#### **Loop Final Para Comparação**

In [ ]:
for nome_modelo, modelo in modelos_numerados.items():
    print("\n" + "="*80)
    print(f" COMPARANDO MODELO: {nome_modelo}")
    print("="*80)

    # Loop pelas classes disponíveis
    for classe in classes:
        print(f"\n Classe: {classe}")

        # Chamando a função de comparação visual
        comparar_modelos_por_classe(
            classe_desejada=classe,
            X_test=X_test,
            y_test=y_test,
            classes=classes,
            resultados={nome_modelo: {"model": modelo}},
            max_imgs=3,
            mostrar_exemplos=True
        )

### **O que está acontecendo**

#### **1. Classes com poucas imagens**

A função `comparar_modelos_por_classe` ignora qualquer classe que possua **menos de 2 imagens no conjunto de teste**.

Isso **não é um erro**, apenas indica que o `X_test` tem poucas amostras para as classes:

* **s1**
* **s2**
* **s6**
* **s8**

**Solução rápida:**
Aumentar o número de imagens no conjunto de teste para essas classes ou duplicar temporariamente algumas amostras apenas para depuração.

---

#### **2. Distâncias muito próximas entre pares iguais e diferentes**

Alguns resultados observados:

```
Mesma classe:       0.000
Classes diferentes: 0.003
```

Ou até inversões, como:

```
Mesma classe > Diferentes
```

Isso mostra que o modelo **não aprendeu uma métrica de distância útil**.
Os embeddings estão **colapsando**, ficando praticamente todos iguais.

---

#### **3. Por que isso está acontecendo?**

As causas mais comuns para esse comportamento são:

1. **Falta de normalização dos embeddings**

   * Sem `K.l2_normalize`, o modelo pode contrair todos os embeddings para um único ponto.

2. **Learning rate muito alto**

   * Para *contrastive loss*, taxas altas tornam a convergência instável.

3. **Pares negativos fáceis demais**

   * Negativos aleatórios não obrigam a rede a aprender limites claros entre classes.

---

#### Interpretação rápida dos resultados por classe

| Classe             | Problema observado                      | Diagnóstico provável                   |
| ------------------ | --------------------------------------- | -------------------------------------- |
| **s1, s2, s6, s8** | Poucas imagens                          | Dataset insuficiente no teste          |
| **s3, s4, s5, s7** | Distâncias inconsistentes               | Embeddings colapsando                  |
| **Geral**          | Distâncias muito próximas ou invertidas | Falta de aprendizado de contraste real |




---

#### **Novos Ajustes**

In [ ]:
# ============================================================
# Rede Siamesa Otimizada com Contrastive Loss
# ============================================================

# ------------------------------------------------------------
# 1. Função de perda Contrastiva
# ------------------------------------------------------------
def contrastive_loss(y_true, y_pred, margin=1.0):
    """
    y_true: 0 = mesma classe, 1 = diferentes
    y_pred: distância euclidiana entre embeddings
    """
    squared_pred = K.square(y_pred)
    margin_term = K.square(K.maximum(margin - y_pred, 0))
    return K.mean((1 - y_true) * squared_pred + y_true * margin_term)


# ------------------------------------------------------------
# 2. Distância Euclidiana entre embeddings
# ------------------------------------------------------------
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


# ------------------------------------------------------------
# 3. Modelo base — extrator de embeddings com L2 normalization
# ------------------------------------------------------------
def criar_modelo_base(input_shape):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dense(128, activation='relu')(x)
    embedding = layers.Dense(64)(x)

    # Normalização L2 — essencial para estabilidade da loss contrastiva
    embedding = layers.Lambda(lambda t: K.l2_normalize(t, axis=1))(embedding)

    return models.Model(inputs, embedding)


# ------------------------------------------------------------
# 4. Rede Siamesa completa
# ------------------------------------------------------------
def criar_rede_siamese(input_shape):
    base_model = criar_modelo_base(input_shape)
    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)

    embedding_a = base_model(input_a)
    embedding_b = base_model(input_b)

    distancia = layers.Lambda(euclidean_distance)([embedding_a, embedding_b])

    siamese_model = models.Model(inputs=[input_a, input_b], outputs=distancia)

    # Learning rate menor para estabilidade
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    siamese_model.compile(loss=contrastive_loss, optimizer=optimizer)

    return siamese_model


# ------------------------------------------------------------
# 5. Gerar pares balanceados (positivos/negativos)
# ------------------------------------------------------------
def gerar_pares_balanceados(X, y, n_pairs_per_class=10):
    pares_a, pares_b, labels = [], [], []
    classes = np.unique(y)
    idx_por_classe = {c: np.where(y == c)[0] for c in classes}

    for c in classes:
        idx_c = idx_por_classe[c]
        if len(idx_c) == 0:
            continue  # pula classe vazia

        for _ in range(n_pairs_per_class):
            # Pares positivos (mesma classe)
            replace_flag = len(idx_c) < 2
            i, j = np.random.choice(idx_c, 2, replace=replace_flag)
            pares_a.append(X[i]); pares_b.append(X[j]); labels.append(0)

            # Pares negativos (classes diferentes)
            c_neg = np.random.choice(classes[classes != c])
            idx_c_neg = idx_por_classe[c_neg]
            i = np.random.choice(idx_c)
            j = np.random.choice(idx_c_neg)
            pares_a.append(X[i]); pares_b.append(X[j]); labels.append(1)

    return np.array(pares_a), np.array(pares_b), np.array(labels)



# ------------------------------------------------------------
# 6. Converter distância → similaridade (para exibição)
# ------------------------------------------------------------
def prever_similaridade(model, img1, img2):
    dist = model.predict([np.expand_dims(img1, 0), np.expand_dims(img2, 0)], verbose=0)
    sim = np.exp(-dist)  # 0 = diferente, 1 = idêntico
    return sim


# ------------------------------------------------------------
# 7. Função para comparação visual de modelos por classe
# ------------------------------------------------------------
def comparar_modelos_por_classe(classe_desejada, X_test, y_test, classes, resultados, max_imgs=5, mostrar_exemplos=True):
    print(f"\nComparando modelos para a classe: {classe_desejada}")

    idx_classe = classes.index(classe_desejada)
    idx_positivos = np.where(y_test == idx_classe)[0]
    idx_negativos = np.where(y_test != idx_classe)[0]

    if len(idx_positivos) < 2:
        print(f"Poucas imagens na classe {classe_desejada} para comparação.")
        return

    X_classe = X_test[np.random.choice(idx_positivos, min(max_imgs, len(idx_positivos)), replace=False)]
    X_outras = X_test[np.random.choice(idx_negativos, min(max_imgs, len(idx_negativos)), replace=False)]

    for i, (run_name, info) in enumerate(resultados.items(), start=1):
        model = info["model"]
        print(f"\n=== Modelo {i} — {run_name} ===")

        dist_pos, dist_neg = [], []

        for a in X_classe:
            for b in X_classe:
                if not np.array_equal(a, b):
                    d = model.predict([a[None, ...], b[None, ...]], verbose=0)[0][0]
                    dist_pos.append(d)

            for b in X_outras:
                d = model.predict([a[None, ...], b[None, ...]], verbose=0)[0][0]
                dist_neg.append(d)

        plt.figure(figsize=(8, 5))
        plt.hist(dist_pos, bins=15, alpha=0.6, label='Mesma Classe', color='green')
        plt.hist(dist_neg, bins=15, alpha=0.6, label='Classes Diferentes', color='red')
        plt.xlabel("Distância (menor = mais similar)")
        plt.ylabel("Frequência")
        plt.title(f"Distribuição de Distâncias — {classe_desejada}\n{run_name}")
        plt.legend()
        plt.show()

        print(f"Distância média — Mesma classe: {np.mean(dist_pos):.3f}")
        print(f"Distância média — Diferentes: {np.mean(dist_neg):.3f}")

        if mostrar_exemplos:
            fig, axes = plt.subplots(2, 3, figsize=(10, 6))
            fig.suptitle(f"Exemplos de Comparação — {classe_desejada} ({run_name})", fontsize=14)
            for j in range(3):
                idx_a, idx_b = np.random.choice(range(len(X_classe)), 2, replace=False)
                a, b = X_classe[idx_a], X_classe[idx_b]
                dist = model.predict([a[None, ...], b[None, ...]], verbose=0)[0][0]
                img_pair = np.hstack((np.squeeze(a), np.squeeze(b)))
                axes[0, j].imshow(img_pair, cmap='gray')
                axes[0, j].set_title(f"Iguais — dist={dist:.3f}")
                axes[0, j].axis("off")

            for j in range(3):
                a = X_classe[np.random.choice(len(X_classe))]
                b = X_outras[np.random.choice(len(X_outras))]
                dist = model.predict([a[None, ...], b[None, ...]], verbose=0)[0][0]
                img_pair = np.hstack((np.squeeze(a), np.squeeze(b)))
                axes[1, j].imshow(img_pair, cmap='gray')
                axes[1, j].set_title(f"Dif. — dist={dist:.3f}")
                axes[1, j].axis("off")
            plt.tight_layout()
            plt.show(block=True)

In [ ]:
pairs_train_a, pairs_train_b, y_train_pairs = gerar_pares_balanceados(X_train, y_train)
pairs_val_a, pairs_val_b, y_val_pairs = gerar_pares_balanceados(X_val, y_val)
pairs_test_a, pairs_test_b, y_test_pairs = gerar_pares_balanceados(X_test, y_test)
input_shape = X_train.shape[1:]
model = criar_rede_siamese(input_shape)
history = model.fit(
    [pairs_train_a, pairs_train_b], y_train_pairs,
    validation_data=([pairs_val_a, pairs_val_b], y_val_pairs),
    epochs=100, batch_size=256
)
modelos_numerados = {"modelo_otimizado": model}

for nome_modelo, modelo in modelos_numerados.items():
    print("\n" + "="*80)
    print(f" COMPARANDO MODELO: {nome_modelo}")
    print("="*80)
    for classe in classes:
        comparar_modelos_por_classe(
            classe_desejada=classe,
            X_test=X_test,
            y_test=y_test,
            classes=classes,
            resultados={nome_modelo: {"model": modelo}},
            max_imgs=3,
            mostrar_exemplos=True
        )

### **Interpretação dos resultados e próximos passos**

#### **Treinamento**

A função de perda apresentou uma boa evolução:
**de 0.55 → 0.07 ao longo de 100 épocas**, enquanto a `val_loss` permaneceu estável em torno de **0.35** durante todo o treinamento.

Essa estabilidade indica que **não há overfitting**, nem explosões de gradiente — um ótimo sinal.
Isso mostra que **o learning rate mais baixo** e a **normalização L2 dos embeddings** estão funcionando como esperado.


#### *Métricas de Distância entre Embeddings*

As novas distâncias médias ficaram assim:

| Classe             | Mesma classe ↓ | Diferentes ↑ | Observação                         |
| ------------------ | -------------- | ------------ | ---------------------------------- |
| **s3**             | 0.262          | 0.183        | Diferença pequena → confusão       |
| **s4**             | 0.098          | 0.187        | Já apresenta separação útil        |
| **s5**             | 0.205          | 0.162        | Embeddings ainda muito próximos    |
| **s7**             | 0.065          | 0.167        | Boa distinção → aprendizagem clara |
| **s1, s2, s6, s8** | poucas imagens | —            | precisam de mais amostras          |


#### **O que isso significa?**

O modelo **já começou a aprender uma noção real de distância**, especialmente nas classes **s4** e **s7**, onde a separação entre pares iguais e diferentes aparece de forma consistente.

Por outro lado, a diferença ainda é pequena em algumas classes. Idealmente, gostaríamos de algo como:

* **Mesma classe:** ~0.1
* **Classes diferentes:** **0.6 ou mais**

Ou seja: o modelo está no caminho certo, mas ainda precisa reforçar melhor os contrastes entre classes — exatamente o que a *contrastive loss* foi feita para ensinar.





---

### **Novo Ajuste**

In [ ]:
# ============================================================
# REDE SIAMESA FINAL OTIMIZADA — META-LEARNING / FEWSHOT
# ============================================================

# ------------------------------------------------------------
# Função de perda Contrastiva com margem ajustável
# ------------------------------------------------------------
def contrastive_loss(y_true, y_pred, margin=2.0):
    """
    y_true: 0 = mesma classe, 1 = diferentes
    y_pred: distância euclidiana entre embeddings
    """
    squared_pred = K.square(y_pred)
    margin_term = K.square(K.maximum(margin - y_pred, 0))
    return K.mean((1 - y_true) * squared_pred + y_true * margin_term)

# ------------------------------------------------------------
# Distância Euclidiana entre embeddings
# ------------------------------------------------------------
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

# ------------------------------------------------------------
# Modelo base com BatchNorm, Dropout e normalização L2
# ------------------------------------------------------------
def criar_modelo_base(input_shape):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)  # 🔸 ajuda a generalizar

    embedding = layers.Dense(64)(x)
    embedding = layers.Lambda(lambda t: K.l2_normalize(t, axis=1))(embedding)

    return models.Model(inputs, embedding)

# ------------------------------------------------------------
# Rede Siamesa completa
# ------------------------------------------------------------
def criar_rede_siamese(input_shape):
    base_model = criar_modelo_base(input_shape)
    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)

    embedding_a = base_model(input_a)
    embedding_b = base_model(input_b)

    distancia = layers.Lambda(euclidean_distance)([embedding_a, embedding_b])
    siamese_model = models.Model(inputs=[input_a, input_b], outputs=distancia)

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    siamese_model.compile(loss=contrastive_loss, optimizer=optimizer)
    return siamese_model

# ------------------------------------------------------------
# Gerador de pares balanceado e robusto
# ------------------------------------------------------------
def gerar_pares_balanceados(X, y, n_pairs_per_class=50):
    pares_a, pares_b, labels = [], [], []
    classes = np.unique(y)
    idx_por_classe = {c: np.where(y == c)[0] for c in classes}

    for c in classes:
        idx_c = idx_por_classe[c]
        if len(idx_c) == 0:
            continue

        for _ in range(n_pairs_per_class):
            # Pares positivos
            replace_flag = len(idx_c) < 2
            i, j = np.random.choice(idx_c, 2, replace=replace_flag)
            pares_a.append(X[i]); pares_b.append(X[j]); labels.append(0)

            # Pares negativos
            c_neg = np.random.choice(classes[classes != c])
            idx_c_neg = idx_por_classe[c_neg]
            i = np.random.choice(idx_c)
            j = np.random.choice(idx_c_neg)
            pares_a.append(X[i]); pares_b.append(X[j]); labels.append(1)

    return np.array(pares_a), np.array(pares_b), np.array(labels)

# ------------------------------------------------------------
# Converter distância → similaridade
# ------------------------------------------------------------
def prever_similaridade(model, img1, img2):
    dist = model.predict([np.expand_dims(img1, 0), np.expand_dims(img2, 0)], verbose=0)
    sim = np.exp(-dist)
    return sim

# ------------------------------------------------------------
# Função de comparação visual por classe
# ------------------------------------------------------------
def comparar_modelos_por_classe(classe_desejada, X_test, y_test, classes, resultados, max_imgs=5, mostrar_exemplos=True):
    print(f"\nComparando modelos para a classe: {classe_desejada}")

    idx_classe = classes.index(classe_desejada)
    idx_positivos = np.where(y_test == idx_classe)[0]
    idx_negativos = np.where(y_test != idx_classe)[0]

    if len(idx_positivos) < 2:
        print(f"Poucas imagens na classe {classe_desejada} para comparação.")
        return

    X_classe = X_test[np.random.choice(idx_positivos, min(max_imgs, len(idx_positivos)), replace=False)]
    X_outras = X_test[np.random.choice(idx_negativos, min(max_imgs, len(idx_negativos)), replace=False)]

    for i, (run_name, info) in enumerate(resultados.items(), start=1):
        model = info["model"]
        print(f"\n=== Modelo {i} — {run_name} ===")

        dist_pos, dist_neg = [], []

        for a in X_classe:
            for b in X_classe:
                if not np.array_equal(a, b):
                    d = model.predict([a[None, ...], b[None, ...]], verbose=0)[0][0]
                    dist_pos.append(d)
            for b in X_outras:
                d = model.predict([a[None, ...], b[None, ...]], verbose=0)[0][0]
                dist_neg.append(d)

        plt.figure(figsize=(8, 5))
        plt.hist(dist_pos, bins=15, alpha=0.6, label='Mesma Classe', color='green')
        plt.hist(dist_neg, bins=15, alpha=0.6, label='Classes Diferentes', color='red')
        plt.xlabel("Distância (menor = mais similar)")
        plt.ylabel("Frequência")
        plt.title(f"Distribuição de Distâncias — {classe_desejada}\n{run_name}")
        plt.legend()
        plt.show()

        print(f"Distância média — Mesma classe: {np.mean(dist_pos):.3f}")
        print(f"Distância média — Diferentes: {np.mean(dist_neg):.3f}")

        if mostrar_exemplos:
            fig, axes = plt.subplots(2, 3, figsize=(10, 6))
            fig.suptitle(f"Exemplos de Comparação — {classe_desejada} ({run_name})", fontsize=14)
            for j in range(3):
                idx_a, idx_b = np.random.choice(range(len(X_classe)), 2, replace=False)
                a, b = X_classe[idx_a], X_classe[idx_b]
                dist = model.predict([a[None, ...], b[None, ...]], verbose=0)[0][0]
                img_pair = np.hstack((np.squeeze(a), np.squeeze(b)))
                axes[0, j].imshow(img_pair, cmap='gray')
                axes[0, j].set_title(f"Iguais — dist={dist:.3f}")
                axes[0, j].axis("off")
            for j in range(3):
                a = X_classe[np.random.choice(len(X_classe))]
                b = X_outras[np.random.choice(len(X_outras))]
                dist = model.predict([a[None, ...], b[None, ...]], verbose=0)[0][0]
                img_pair = np.hstack((np.squeeze(a), np.squeeze(b)))
                axes[1, j].imshow(img_pair, cmap='gray')
                axes[1, j].set_title(f"Dif. — dist={dist:.3f}")
                axes[1, j].axis("off")
            plt.tight_layout()
            plt.show(block=True)

# ------------------------------------------------------------
# Treinamento e avaliação
# ------------------------------------------------------------
pairs_train_a, pairs_train_b, y_train_pairs = gerar_pares_balanceados(X_train, y_train, n_pairs_per_class=50)
pairs_val_a, pairs_val_b, y_val_pairs = gerar_pares_balanceados(X_val, y_val, n_pairs_per_class=30)
pairs_test_a, pairs_test_b, y_test_pairs = gerar_pares_balanceados(X_test, y_test, n_pairs_per_class=20)

input_shape = X_train.shape[1:]
model = criar_rede_siamese(input_shape)

history = model.fit(
    [pairs_train_a, pairs_train_b], y_train_pairs,
    validation_data=([pairs_val_a, pairs_val_b], y_val_pairs),
    epochs=100, batch_size=256
)

# ------------------------------------------------------------
# 9️ Visualização dos resultados
# ------------------------------------------------------------
modelos_numerados = {"modelo_final_otimizado": model}

for nome_modelo, modelo in modelos_numerados.items():
    print("\n" + "="*80)
    print(f" COMPARANDO MODELO: {nome_modelo}")
    print("="*80)
    for classe in classes:
        comparar_modelos_por_classe(
            classe_desejada=classe,
            X_test=X_test,
            y_test=y_test,
            classes=classes,
            resultados={nome_modelo: {"model": modelo}},
            max_imgs=3,
            mostrar_exemplos=True
        )

# ------------------------------------------------------------
# Visualização dos embeddings com t-SNE
# ------------------------------------------------------------
base_model = model.layers[2]  # extrai o submodelo de embeddings
embeddings = base_model.predict(X_test)

tsne = TSNE(n_components=2, perplexity=10, random_state=42)
emb_2d = tsne.fit_transform(embeddings)

plt.figure(figsize=(8,6))
sns.scatterplot(x=emb_2d[:,0], y=emb_2d[:,1], hue=y_test, palette='tab10', s=60)
plt.title("Visualização dos embeddings (t-SNE)")
plt.xlabel("Dimensão 1")
plt.ylabel("Dimensão 2")
plt.legend(title="Classes")
plt.show()

### **ANÁLISE DOS RESULTADOS — MODELO FINAL OTIMIZADO**

O modelo foi treinado por 100 épocas com batch size = 256, normalização L2 e margem = 2.0.
As curvas de loss indicaram convergência estável e contínua:

    Loss inicial:     ~1.02
    Val_loss inicial: ~1.54
    Loss final:       ~0.38
    Val_loss final:   ~0.73

 INTERPRETAÇÃO GERAL:
- O modelo aprendeu uma métrica de similaridade real — não há colapso de embeddings.
- As perdas diminuíram gradualmente, sem sinais de overfitting.
- A normalização e o aumento de pares balanceados melhoraram a estabilidade do treino.

#### **RESULTADOS POR CLASSE (distâncias médias)**

| Classe             | Mesma Classe ↓ | Diferentes ↑ | Interpretação                                                    |
| ------------------ | -------------- | ------------ | ---------------------------------------------------------------- |
| **s3**             | 1.453          | 1.450        | Distâncias quase iguais → embeddings ainda pouco discriminativos |
| **s4**             | 1.047          | 1.645        | Boa separação → métrica aprendida com sucesso                    |
| **s5**             | 1.307          | 1.070        | Inversão parcial → confusão entre classes semelhantes            |
| **s7**             | 1.782          | 1.231        | Boa separação, mas distâncias gerais muito altas                 |
| **s1, s2, s6, s8** | —              | —            | Poucas imagens → não comparadas                                  |


 **CONCLUSÃO:**
- O modelo estabilizou o aprendizado e diferencia melhor pares semelhantes e distintos.
- Ainda há dispersão alta no espaço de embeddings (distâncias entre 1.0 e 1.7).
- Classes com menos amostras precisam ser reforçadas no conjunto de teste.

 **PRÓXIMOS PASSOS:**

1. Reduzir a margem da contrastive loss (ex: margin=1.0) → melhora compactação dos clusters.
2. Normalizar a camada de distância (limitando valores entre 0 e 1).
3. Treinar por mais 50–100 épocas, pois a loss ainda apresentava tendência de queda.
4. Gerar mais pares por classe para diversificar amostras e reduzir viés.
5. Visualizar os embeddings com t-SNE para confirmar a separação espacial entre classes.

 **Em resumo:**
O modelo passou da fase de aprendizado básico (colapso de embeddings) para um estágio de refinamento da métrica de similaridade.
A estrutura atual é funcional, e os ajustes seguintes devem focar em aperfeiçoar a separação entre classes no espaço vetorial.



---

### **Valiação de Similaridade com Múltiplas Imagens de Referência por Classe:**
Para avaliar a robustez da métrica de similaridade no espaço de embeddings, calculou-se a similaridade média entre cada imagem do conjunto de teste e um conjunto de múltiplas imagens de referência por classe. O objetivo é verificar se o modelo consegue manter alta auto-similaridade intra-classe e baixa similaridade inter-classe, mesmo quando utiliza um conjunto diversificado de exemplos de suporte.

In [ ]:
# ------------------------------------------------------------
# Selecionar múltiplas imagens de referência por classe
# ------------------------------------------------------------
def selecionar_referencias_por_classe(X_ref, y_ref, classes, n_ref=5, random_state=42):
    """
    Retorna um dicionário:
        {nome_classe: array_indices_no_X_ref}
    escolhendo até n_ref imagens por classe como "suporte" (referência).
    """
    rng = np.random.RandomState(random_state)
    idx_refs = {}

    for idx_cls, nome_cls in enumerate(classes):
        indices_da_classe = np.where(y_ref == idx_cls)[0]
        if len(indices_da_classe) == 0:
            continue

        n_escolher = min(n_ref, len(indices_da_classe))
        escolhidos = rng.choice(indices_da_classe, size=n_escolher, replace=False)
        idx_refs[nome_cls] = escolhidos

    return idx_refs

# ------------------------------------------------------------
# Extrair embeddings das imagens de referência
# ------------------------------------------------------------
def construir_embeddings_referencia(base_model, X_ref, idx_refs):
    """
    Usa o base_model (rede de embeddings) para projetar as imagens de referência
    no espaço latente. Retorna:
        {nome_classe: matriz [n_ref, dim_embedding]}
    """
    emb_refs = {}
    for nome_cls, indices in idx_refs.items():
        emb = base_model.predict(X_ref[indices], verbose=0)
        emb_refs[nome_cls] = emb
    return emb_refs

# ------------------------------------------------------------
# Calcular similaridade média de uma imagem para cada classe
# ------------------------------------------------------------
def similaridade_media_para_classes(emb_query, emb_refs_por_classe):
    """
    Dado um embedding de consulta (emb_query) e um dicionário com os embeddings
    de referência de cada classe, calcula a similaridade média da imagem de
    consulta com cada classe, usando:
        distância = norma L2
        similaridade = exp(-distância)
    Retorna um dicionário:
        {nome_classe: similaridade_media}
    """
    sims = {}
    for nome_cls, emb_refs in emb_refs_por_classe.items():

        dists = np.linalg.norm(emb_refs - emb_query, axis=1)
        sim = np.exp(-dists)
        sims[nome_cls] = sim.mean()
    return sims

# ------------------------------------------------------------
# Avaliar similaridade média em todo o conjunto de teste
# ------------------------------------------------------------
def avaliar_similaridade_multireferencia(base_model, emb_refs_por_classe,
                                         X_test, y_test, classes,
                                         max_por_classe=None):
    """
    Percorre o conjunto de teste e calcula:
        - similaridade média com a classe correta (same_class_sims)
        - similaridade média com classes incorretas (diff_class_sims)
    Também guarda estatísticas por classe em um dicionário.

    max_por_classe: se não for None, limita a quantidade de imagens de teste
    usadas por classe (só pra não ficar muito pesado).
    """
    same_class_sims = []
    diff_class_sims = []
    por_classe = {nome: {"same": [], "diff": []} for nome in emb_refs_por_classe.keys()}

    indices_para_usar = np.arange(len(X_test))
    if max_por_classe is not None:
        indices_para_usar = []
        for idx_cls, nome_cls in enumerate(classes):
            idxs = np.where(y_test == idx_cls)[0]
            if len(idxs) == 0:
                continue
            n = min(max_por_classe, len(idxs))
            chosen = np.random.choice(idxs, size=n, replace=False)
            indices_para_usar.extend(chosen)
        indices_para_usar = np.array(indices_para_usar)

    for i in indices_para_usar:
        img = X_test[i]
        label_idx = y_test[i]
        nome_true = classes[label_idx]


        emb_query = base_model.predict(img[None, ...], verbose=0)[0]

        sims = similaridade_media_para_classes(emb_query, emb_refs_por_classe)

        for nome_cls, sim_media in sims.items():
            if nome_cls == nome_true:
                same_class_sims.append(sim_media)
                por_classe[nome_cls]["same"].append(sim_media)
            else:
                diff_class_sims.append(sim_media)
                por_classe[nome_cls]["diff"].append(sim_media)

    same_class_sims = np.array(same_class_sims)
    diff_class_sims = np.array(diff_class_sims)

    print(f"Similaridade média (global) — MESMA classe:    {same_class_sims.mean():.3f}")
    print(f"Similaridade média (global) — CLASSES DIFER.: {diff_class_sims.mean():.3f}")

    plt.figure(figsize=(8, 5))
    plt.hist(same_class_sims, bins=20, alpha=0.6, label='Mesma classe')
    plt.hist(diff_class_sims, bins=20, alpha=0.6, label='Classes diferentes')
    plt.xlabel("Similaridade média (exp(-distância))")
    plt.ylabel("Frequência")
    plt.title("Distribuição da similaridade média usando múltiplas imagens de referência")
    plt.legend()
    plt.show()

    print("\n--- Estatísticas por classe ---")
    for nome_cls, d in por_classe.items():
        if len(d["same"]) == 0:
            continue
        print(f"\nClasse: {nome_cls}")
        print(f"  média (same) = {np.mean(d['same']):.3f}  |  desvio = {np.std(d['same']):.3f}")
        if len(d["diff"]) > 0:
            print(f"  média (diff) = {np.mean(d['diff']):.3f}  |  desvio = {np.std(d['diff']):.3f}")

    return same_class_sims, diff_class_sims, por_classe


In [ ]:
# 1) Escolher, por exemplo, 5 imagens de referência por classe no conjunto de treino
idx_refs = selecionar_referencias_por_classe(
    X_ref=X_train,
    y_ref=y_train,
    classes=classes,
    n_ref=5,
    random_state=42
)

# 2) Construir embeddings das referências
emb_refs_por_classe = construir_embeddings_referencia(
    base_model=base_model,
    X_ref=X_train,
    idx_refs=idx_refs
)

# 3) Avaliar no conjunto de teste
same_sims, diff_sims, stats_por_classe = avaliar_similaridade_multireferencia(
    base_model=base_model,
    emb_refs_por_classe=emb_refs_por_classe,
    X_test=X_test,
    y_test=y_test,
    classes=classes,
    max_por_classe=50
)

Nesta etapa, foi implementado um procedimento para avaliar a métrica aprendida pela rede no espaço de *embeddings*, utilizando múltiplas imagens de referência por classe. Inicialmente, são selecionados diversos exemplos de suporte para cada classe e, em seguida, seus *embeddings* são gerados pelo modelo base. Para cada imagem do conjunto de teste, obtém-se o *embedding* correspondente e calcula-se sua distância para todos os *embeddings* de referência de cada classe, convertendo essas distâncias em medidas de similaridade por meio da função.


A similaridade média com cada classe é então utilizada para comparar, estatisticamente, a proximidade entre amostras da mesma classe e de classes distintas. Por fim, analisam-se as distribuições de similaridade — globais e por classe — permitindo avaliar o grau de separação entre as classes no espaço latente aprendido pelo modelo.

### **RESULTADOS OBTIDOS**

- **Similaridade média** — mesma classe: 0.392

- **Similaridade média** — classes diferentes: 0.329

Esses valores indicam uma separação moderada entre as distribuições. Embora a diferença não seja grande, observa-se uma tendência de que amostras da mesma classe apresentem maior similaridade entre si, sugerindo que o modelo internalizou alguma estrutura interclasse. Ainda assim, a fronteira entre as classes permanece difusa.

O histograma reforça esse comportamento:

- as distribuições se sobrepõem significativamente;

- porém há um deslocamento perceptível da curva de “mesma classe” para valores mais altos.

### **Análise por Classe**

#### **Classes com melhor separação**

**s1:** 0.591 (mesma) | 0.333 (diferentes)
**s2:** 0.559 (mesma) | 0.352 (diferentes)

Essas classes apresentam uma diferença nítida entre similaridade intra-classe e inter-classe, indicando que formaram **clusters bem definidos** no espaço de embeddings.

---

#### **Classes com separação moderada**

**s3:** 0.474 vs 0.373
**s5:** 0.497 vs 0.338
**s6:** 0.456 vs 0.390

Aqui a separação existe, mas é **relativamente pequena**. As distribuições ainda se sobrepõem de maneira considerável, sugerindo que essas classes estão **parcialmente confundidas** com classes próximas.

---

#### **Classes com comportamento invertido ou instável**

**s7:** 0.182 vs 0.238
**s8:** 0.159 vs 0.378

Nessas classes, a similaridade média com outras classes é maior do que com elas mesmas — um comportamento **invertido**, indicando que o modelo está tendo dificuldade em posicioná-las corretamente no espaço latente.

As causas mais prováveis incluem:

* baixa variabilidade nos exemplos dessas classes;
* embeddings pouco estruturados ou colapsados;
* padrões visuais muito parecidos com os de outras classes.

---
